# Global Configuration

In [1]:
# Is_To_Run_On_Cloud = True
from keras import backend as K
import keras
import json
# Is_HPC = True
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# if(Is_HPC):
#     pass
# else:
import pandas as pd
import tensorflow as tf

import numpy as np
from keras.layers.core import Flatten, Dense,Dropout
from keras import Model
from keras.preprocessing.image import  ImageDataGenerator
from keras.utils import multi_gpu_model
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
import time
from time import sleep
import gc
from datetime import datetime
from enum import Enum
import os
import cv2
from tensorflow.python.client import device_lib
# if(Is_To_Run_On_Cloud):
#     from keras import keras_applications as K_Models
# else:
from keras import applications as K_Models
# Is_Running_On_Local_Desktop = True
# FAA_HPC = False
from classification_models.resnet import ResNet18, ResNet34, preprocess_input

    
# if(Training_Gauges["True Airspeed(kn)"])

/home/hikmat/anaconda3/envs/KerasWithTFPY35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
Config_CSV_File_Name = "Guages_Training"
if(os.path.isfile("./{0}.csv".format(Config_CSV_File_Name))):
    Training_Gauges_CSV = pd.read_csv("./{0}.csv".format(Config_CSV_File_Name))
#     print("Loaded CSV Path=", "./{0}_Prepared_MetaData.csv".format(FAA_PREFIX))
    print("File found.")
else:
    print("Guages_Inference.csv does not exist.")
Gauges = []
Gauges = ["True Airspeed(kn)", 
    "NR(%)",
    "N1_NG 1(%)",
    "N1_NG 2(%)"
         ]
print("Model will be trained for the following gauges.")

File found.
Model will be trained for the following gauges.


In [3]:
PADDING = "SAME" # "VALID"
def LenNet(input_size, output_classes):
    # Vanilla CNN. Play around with architecture.
    model = Sequential()
    model.add(Conv2D(filters = 64, 
                     kernel_size = 9, 
                     strides = 1, 
                     activation = 'relu', 
                     padding=PADDING,
                     input_shape = (input_size[0],input_size[1],input_size[2])))
    model.add(Conv2D(filters = 64, 
                     kernel_size = 7, 
                     strides = 1, 
                     activation = 'relu',
                     padding=PADDING,
                    ))
    model.add(Conv2D(filters = 64, 
                     kernel_size = 5, 
                     strides = 1, 
                     activation = 'relu', 
                     padding=PADDING,
                     ))
    model.add(MaxPooling2D(pool_size = 5, strides = 1))
    model.add(Conv2D(filters = 64, 
                 kernel_size = 3,
                 strides = 1,
                 activation = 'relu',
                 padding=PADDING,
                ))
    model.add(Conv2D(filters = 64, 
                 kernel_size = 3,
                 strides = 1,
                 padding=PADDING,
                 activation = 'relu'
                 ))
    model.add(Conv2D(filters = 64, 
                 kernel_size = 3,
                 strides = 1,
                 padding=PADDING,
                 activation = 'relu'
                 ))
    model.add(MaxPooling2D(pool_size = 5, strides = 1))
    model.add(Conv2D(filters = 64, 
                     kernel_size = 3,
                     strides = 1,
                     activation = 'relu',
                     padding=PADDING
                     ))
    model.add(Conv2D(filters = 64, 
                     kernel_size = 3,
                     strides = 1,
                     activation = 'relu',
                     padding=PADDING
                    ))
    model.add(MaxPooling2D(pool_size = 3, strides = 2))
    model.add(Dropout(0.25))
    model.add(Flatten())
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(output_classes, activation='softmax', name='softmax'))
    return model

In [4]:
############################################################################################## 
def Load_Model():
    global ML_Model
    if(SELECTED_MODEL == Available_Models.VGG16):
        ML_Model = K_Models.VGG16(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.VGG19):
        ML_Model = K_Models.VGG19(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.InceptionV3):
        ML_Model = K_Models.InceptionV3(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.ResNet50):

        ML_Model = K_Models.ResNet50(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
        
    elif(SELECTED_MODEL == Available_Models.ResNet18):
        ML_Model = ResNet18(input_shape=INPUT_SIZE, weights=INITIALIZATION_WIEGHTS, include_top=INCLUDE_TOP)
        
    elif(SELECTED_MODEL == Available_Models.ResNet34):
        ML_Model = ResNet34(input_shape=INPUT_SIZE, weights=INITIALIZATION_WIEGHTS, include_top=INCLUDE_TOP)
        
    elif(SELECTED_MODEL == Available_Models.DenseNet121):
        ML_Model = K_Models.DenseNet121(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.DenseNet169):
        ML_Model = K_Models.DenseNet169(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.DenseNet201):
        ML_Model = K_Models.DenseNet201(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.MobileNet):
        ML_Model = K_Models.MobileNet(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)

    elif(SELECTED_MODEL == Available_Models.InceptionResNetV2):
        ML_Model = K_Models.InceptionResNetV2(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.NASNetLarge):
        ML_Model = K_Models.NASNetLarge(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                            pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.Xception):
        ML_Model = K_Models.Xception(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.NASNetMobile):
        ML_Model = K_Models.NASNetMobile(include_top=INCLUDE_TOP, weights=INITIALIZATION_WIEGHTS, 
                                                              input_tensor=INPUT_TENSOR, input_shape=INPUT_SIZE,
                                                              pooling=POOLING, classes=OUTPUT_CLASSES)
    elif(SELECTED_MODEL == Available_Models.Lenet_Style):
        ML_Model = LenNet(input_size= INPUT_SIZE, output_classes=OUTPUT_CLASSES)
        return
    else:
        print("NO MODEL IDENTIFIER MATCHED")
        
   
    if(POOLING is None):
        last_layer = Flatten()(ML_Model.output)
    else:
        last_layer = ML_Model.output
        if(SELECTED_MODEL ==  Available_Models.ResNet18 or SELECTED_MODEL == Available_Models.ResNet34):
            last_layer = keras.layers.GlobalAveragePooling2D()(ML_Model.output)
        
    if(len(Dense_Layers_Neurons) > 0):
        for n in Dense_Layers_Neurons:
            last_layer = Dense(n, activation='relu')(last_layer)
            if(Is_To_Add_Dropout):
                last_layer =  Dropout(Dropout_Rate)
    else:
        if(Is_To_Add_Dropout):
    #         for neurons in Dense_Layers:
    #             last_layer = Dense(neurons, activation='relu')(last_layer)
            last_layer =  Dropout(Dropout_Rate)(last_layer)
        
    last_layer = Dense(OUTPUT_CLASSES, activation=FINAL_LAYER_CLASSIFICATION_FUCNTION,
                       name='softmax')(last_layer)
    ML_Model = Model(ML_Model.input, last_layer)
##############################################################################################



##############################################################################################
def Freeze_Model_Parameters():#ML_Model, FREEZE_EARLY_LAYERS_WIHT_PERCENT):
    global ML_Model
    if(FREEZE_EARLY_LAYERS_WIHT_PERCENT <= 0):
        print("Zero layers freezed")
    else:
        # Freezing Models Layers
        freezed_layers_count = int(len(ML_Model.layers) * FREEZE_EARLY_LAYERS_WIHT_PERCENT)
        print("Freeze {0} early layers".format(freezed_layers_count))
        for layer in ML_Model.layers[:freezed_layers_count]:
            layer.trainable = False
    

##############################################################################################



##############################################################################################
def Add_Conv_Regularizer():#ML_Model, FREEZE_EARLY_LAYERS_WIHT_PERCENT):
    global ML_Model
#     alpha = 0.0001  # weight decay coefficient
    if(REGULARIZE_EARLY_LAYERS_WIHT_PERCENT <= 0):
        print("Zero layers regularized")
    else:
        # Freezing Models Layers
        regularized_layers_count = int(len(ML_Model.layers) * REGULARIZE_EARLY_LAYERS_WIHT_PERCENT)
        print("Regularize {0} early layers".format(regularized_layers_count))
        for layer in ML_Model.layers[:regularized_layers_count]:
            
            if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
#                 print("Before Regularization Added=" + str(layer.losses))
                layer.add_loss(keras.regularizers.l2(REGULARIZE_EARLY_LAYERS_WITH_RATIO)(layer.kernel))
#                 print("Layer Name=", layer.name)
#                 print("After Regularization Added=" + str(layer.losses))
            if hasattr(layer, 'bias_regularizer') and layer.use_bias:
                layer.add_loss(keras.regularizers.l2(REGULARIZE_EARLY_LAYERS_WITH_RATIO)(layer.bias))
            
    

##############################################################################################



##############################################################################################
def Compile_Model():#(ML_Model, LEARNING_RATE):
    global P_ML_Model
    global ML_Model

    if(AVAILABLE_GPUs > 1):
        print("Compiling multi GPU models")
        P_ML_Model = multi_gpu_model(ML_Model, AVAILABLE_GPUs)
    else:
        print("Compiling single GPU model")
        P_ML_Model = ML_Model

    adam = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999,
            epsilon=None, decay=1e-6, amsgrad=False)
    P_ML_Model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=['accuracy'])
    
##############################################################################################



##############################################################################################
def Create_Data_Loader():
    global TRAIN_GENERATOR
    global VALID_GENERATOR
    global STEP_SIZE_TRAIN
    global STEP_SIZE_VALID
    
    if(Is_To_Apply_Data_Augmentation):
        #Observation: Data augmentation didn't help with guages.
        train_datagen = ImageDataGenerator(
            rescale=1./255,
#             rotation_range=40,
#             width_shift_range=0.2,
#             height_shift_range=0.2,
#             shear_range=0.2,
#             zoom_range=[-0.2, 0.2],
#             horizontal_flip=True,
#             fill_mode='nearest'
        )
        validation_datagen = ImageDataGenerator(
            rescale=1./255)
    else:
        train_datagen = ImageDataGenerator(rescale=1./255)
        validation_datagen = ImageDataGenerator(rescale=1./255)

    TRAIN_GENERATOR = train_datagen.flow_from_directory(TRAINING_DIRECTORY_PATH,  # this is the target directory
                classes =CLASS_MAPPINGS,
                target_size=(INPUT_SIZE[0], INPUT_SIZE[1]),  # all images will be resized 
                batch_size=BATCH_SIZE,
                color_mode=COLOR_MODE,
                shuffle=SHUFFLE,
                seed=SEED,
                class_mode='categorical') 
    print("TRAINING_DIRECTORY_PATH=",TRAINING_DIRECTORY_PATH)
    print("TRAIN_GENERATOR.class_indices =", TRAIN_GENERATOR.class_indices)
    
    VALID_GENERATOR = validation_datagen.flow_from_directory(
                VALIDATION_DIRECTORY_PATH,
                classes =CLASS_MAPPINGS,
                target_size=(INPUT_SIZE[0], INPUT_SIZE[1]),
                batch_size=BATCH_SIZE,
                color_mode=COLOR_MODE,
                seed=SEED,
                class_mode='categorical')
    print("VALIDATION_DIRECTORY_PATH=",VALIDATION_DIRECTORY_PATH)
    print("TRAIN_GENERATOR.class_indices =", VALID_GENERATOR.class_indices)

    STEP_SIZE_TRAIN=TRAIN_GENERATOR.n//TRAIN_GENERATOR.batch_size
    STEP_SIZE_VALID=VALID_GENERATOR.n//VALID_GENERATOR.batch_size
    print("STEP_SIZE_TRAIN=",STEP_SIZE_TRAIN)
    print("STEP_SIZE_VALID=",STEP_SIZE_VALID)
##############################################################################################    



##############################################################################################
def Get_Model_Storage_Info():
    global Model_Name
    global Weights_Path
    global History_Path
    global Model_Path
#     Model_Name = "{0}_{1}_Degree_{2}_With_D_Aug_{3}".format(
#         MODEL_NAME_PREFIX, SELECTED_MODEL, C_Guage, Is_To_Apply_Data_Augmentation)
    Model_Name = "{0}_{1}".format(
        SELECTED_MODEL, C_Guage)
    
#     Weights_Path = "{0}/CheckPoint_{1}_Best_Model_Weights_With_D_Aug_{2}.hdf5".format(Store_Trained_ML_Model_Info_At, 
#                                                                        Model_Name, Is_To_Apply_Data_Augmentation)
    Weights_Path = "{0}/CheckPoint_{1}.hdf5".format(
                    Store_Trained_ML_Model_Info_At, Model_Name)
    
#     History_Path = "{0}/{1}_history_With_D_Aug_{2}".format(Store_Trained_ML_Model_Info_At, 
#                                             Model_Name, Is_To_Apply_Data_Augmentation)
    History_Path = "{0}/{1}_history".format(Store_Trained_ML_Model_Info_At, 
                                            Model_Name)
#     Model_Path = "{0}/{1}_model_architecture_With_D_Aug_{2}".format(Store_Trained_ML_Model_Info_At,
#                                                           Model_Name, Is_To_Apply_Data_Augmentation)
    Model_Path = "{0}/{1}".format(Store_Trained_ML_Model_Info_At,
                                                          Model_Name)
    print("Model_Name=", Model_Name)
    print("Weights_Path=", Weights_Path)
    print("History_Path=", History_Path)
    print("Model_Path=", Model_Path)
##############################################################################################



##############################################################################################
def Do_Training():#(ML_Model,P_ML_Model,train_generator,validation_generator,STEP_SIZE_TRAIN, STEP_SIZE_VALID, 
                #EPOCHs, CLASSES_WEIGTHAGES, weights_path):

    global History
    #lr_scheduler = LearningRateScheduler(lr_schedule)
    checkpoint = ModelCheckpoint(Weights_Path, monitor=Monitor, verbose=1, save_best_only=True, mode=Mode)
    early_stopping = EarlyStopping(monitor=Monitor, min_delta=0, patience=EARLY_STOPPING_PATIENCE, verbose=0, 
                                       mode=Mode)

    #     lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
    print("CLASSES_WEIGTHAGES=", CLASSES_WEIGTHAGES)
    callbacks_list = [early_stopping,  checkpoint]#, lr_scheduler]
    History = P_ML_Model.fit_generator(
            TRAIN_GENERATOR,
            steps_per_epoch=STEP_SIZE_TRAIN,#FAA_TRAINING_EXAMPLES // FAA_BATCH_SIZE,
            epochs=EPOCHs,
            validation_data=VALID_GENERATOR,
            validation_steps=STEP_SIZE_VALID, #FAA_VALIDATION_EXAMPLES // FAA_BATCH_SIZE,
             callbacks=callbacks_list,
            class_weight = CLASSES_WEIGTHAGES)
##############################################################################################



##############################################################################################
def Save_Model_Info():
    with open('{0}.json'.format(History_Path), 'w') as f:
        json.dump(History.history, f)
#     if(Is_HPC):
#         print("NO PANDA INSTALLATION")
#     else:
    pd.DataFrame(History.history).to_csv("{0}.csv".format(History_Path))
    try:
        model_yaml = ML_Model.to_yaml()
        with open("{0}.yaml".format(Model_Path), "w") as yaml_file:
            yaml_file.write(model_yaml)
    except Exception as eExcep:
        print(str(eExcep))
    model_json = ML_Model.to_json()
    with open("{0}.json".format(Model_Path), "w") as json_file:
        json_file.write(model_json)
#     ML_Model.save(Model_Path)
##############################################################################################



##############################################################################################
def Evaluate_Model():
    
    global TEST_CSV
    try:
        TEST_CSV["{0}_Prd_{1}_Selected_Model_{2}".format(C_Guage,Model_Name, SELECTED_MODEL)] = None
        try:
            for index, row in TEST_CSV.iterrows():
#                 try:
                if(index != 0 and index % 2500 == 0):
                    print("Predicted Examples Count={0}".format(index))
#                     break
                for test_img_path in TEST_CSV["ML_File_Path_For_TrainValTest"][index].split(","):
#                     print("test_img_path=", test_img_path)
                    try:
                        test_img_path = test_img_path.replace("./", "{0}/".format(Dell_Path))
    #                     print("test_img_path=", test_img_path)
                        test_img = cv2.imread(test_img_path, cv2.IMREAD_COLOR)
                        test_img = test_img.astype("float32") / 255
                        #                     test_img = np.expand_dims(test_img, -1)
                        test_img = cv2.resize(test_img, (INPUT_SIZE[0], 
                                                           INPUT_SIZE[1]))
                        test_img = np.reshape(test_img,(1,test_img.shape[0], 
                                    test_img.shape[1], test_img.shape[2]))
                        prediction = np.argmax(ML_Model.predict(test_img, 
                                                            batch_size=1))
                        TEST_CSV["{0}_Prd_{1}_Selected_Model_{2}".format(C_Guage,Model_Name, SELECTED_MODEL)][index] = prediction
                        #                 if(TEST_CSV["{0}_Prd_{1}".format(C_Guage,Model_Name)] = None)
                        #                 TEST_CSV["{0}_Prd_{1}".format(C_Guage,Model_Name)][index] = prediction
    #                     print("Prediction=" + str(prediction))
                    except Exception as eERRoR:
                        TEST_CSV["{0}_Prd_{1}_Selected_Model_{2}".format(C_Guage,Model_Name, SELECTED_MODEL)][index] = -1
                    break
#                 except Exception as eError:
#                     TEST_CSV["{0}_Prd_{1}".format(C_Guage, Model_Name)][index] = "Error"
#                     print("Some Error Occurred While Prediction." + str(eError))
#                     continue
        except Exception as eError:
            TEST_CSV["{0}_Prd_{1}_Selected_Model_{2}".format(C_Guage,Model_Name, SELECTED_MODEL)][index] = "Error"
            print("Some Error Occurred While Prediction." + str(eError))

        
    except Exception as eError:
        print("Some Error Occurred While Evaluating the Model." + str(eError))
    
                
    
##############################################################################################




##############################################################################################
def Initiate_Model_Training():
#     print("SELECTED MODEL #" + str(Selected_Model))
    global ML_Model
    global P_ML_Model
    
    print("SELECTED MODEL #" + str(SELECTED_MODEL))
    if(AVAILABLE_GPUs > 1):
        with tf.device('/cpu:0'):
            Load_Model()
    else:
        Load_Model()
    Add_Conv_Regularizer()
    Freeze_Model_Parameters()#(ML_Model, FREEZE_EARLY_LAYERS_WIHT_PERCENT)
    Compile_Model()
    Create_Data_Loader()
    Get_Model_Storage_Info()
    ML_Model.summary()
    Do_Training()
    Save_Model_Info()
#     Evaluate_Model()
    #Release Resources
    ML_Model = None
    P_ML_Model = None
    gc.collect()
    K.clear_session()
    sleep(COOL_DOWN)
##############################################################################################

In [5]:
# AVAILABLE_GPUs = len([gpu for gpu in device_lib.list_local_devices() if gpu.device_type=="GPU"])
# print(AVAILABLE_GPUs)

In [6]:
# Global Configurations
# Hyper-parameters Configuration etc

##############################################################################################
#HORIZON MASK OR NOT MASK SETTING
# With_Horizon = None
# Horizon = None
# # Is_Horizon_Masked = False
# Degree_Constraints = [3, 4]#, 5, 6]#, 5, 6]#[3, 4, 5, 6]
# C_Degree = None # Current Degree Under Training
C_Guage = None
# C_With_Horizon = None
# if(Is_Horizon_Masked):
#     With_Horizon = "With_Horizon_Masked" 
# else:
#     With_Horizon = "Without_Horizon_Masked" 
# Horizon = [ "Without_Horizon_Masked", "With_Horizon_Masked",]
# print("Horizon=", Horizon)
##############################################################################################


##############################################################################################
# ML MODEL
#SINGULARITY (Global Variables)
# MODEL PARAMETERS
#LOCAL MACHINE IS RUNNING MAX VALIDATION ACCURACCY EXP
Monitor = "val_acc" # Monitor for maximium val_acc or Minimium val_loss
Mode = "auto" # max for val_acc and min for val_loss
COOL_DOWN = 25
ML_Model = None
P_ML_Model = None
TRAIN_GENERATOR = None
VALID_GENERATOR = None
STEP_SIZE_TRAIN = None
STEP_SIZE_VALID = None
Model_Name = None
Weights_Path = None
History_Path = None
Model_Path = None
History = None
SEED = 101
OUTPUT_CLASSES = 9
AVAILABLE_GPUs = len([gpu for gpu in device_lib.list_local_devices() if gpu.device_type=="GPU"])
if(AVAILABLE_GPUs == 0):
    AVAILABLE_GPUs = 1
    print("AVAILABLE_GPUs = 0 (NO GPU ACCESS)")
print("AVAILABLE_GPUs=",AVAILABLE_GPUs)

FINE_TUNE = False


EARLY_STOPPING_PATIENCE = 5
COLOR_MODE = "rgb"
FINAL_LAYER_CLASSIFICATION_FUCNTION = "softmax"
SHUFFLE = True  
POOLING = None#"avg" # avg or max or None for flatten
INITIALIZATION_WIEGHTS = "imagenet"
INCLUDE_TOP = False
FREEZE_EARLY_LAYERS_WIHT_PERCENT = 0
INPUT_SIZE = (224,224, 3)
INPUT_TENSOR = None
BATCH_SIZE = 0 # Must be zero here 
EPOCHs = 500


# 25000 Classes
CLASSES_WEIGTHAGES = None
#{0: 1.0, 1: 1.9471826700742363, 2: 1.0, 
                      #3: 1.0, 4: 1.8986590720303786, 5: 1.7509301816590064,
                      #6: 7.0609002647837595, 7: 10.256410256410257, 8: 1.0}


######################################
# Is_To_Apply_Class_Weigtage = False
#Degree 3 Constrant On P & R
# Classwise_Dist_Degree_3= {0: 16000, 1: 8217, 2: 16000, 3: 16000, 4: 8427, 5: 9138, 6: 2266, 7: 1560, 8: 16000}
# Classwise_Weightage_Degree_3= {0: 1.0, 1: 1.9471826700742363, 2: 1.0, 3: 1.0, 4: 1.8986590720303786, 5: 1.7509301816590064, 
#                                6: 7.0609002647837595, 7: 10.256410256410257, 8: 1.0}

######################################
######################################
# Classwise_Dist_Degree_4= {0: 16000, 1: 3034, 2: 16000, 3: 16000, 4: 4486, 5: 3702, 6: 953, 7: 417, 8: 16000}
# Classwise_Weightage_Degree_4= {0: 1.0, 1: 5.273566249176005, 2: 1.0, 3: 1.0, 4: 3.5666518056174765, 5: 4.321988114532685, 
#                                6: 16.7890870933893, 7: 38.36930455635492, 8: 1.0}
######################################
######################################
# Classwise_Dist_Degree_5= {0: 14316, 1: 1502, 2: 16000, 3: 16000, 4: 2059, 5: 1448, 6: 572, 7: 131, 8: 16000}
# Classwise_Weightage_Degree_5= {0: 1.1176306230790725, 1: 10.652463382157125, 2: 1.0, 3: 1.0, 4: 7.770762506070908, 5: 11.049723756906078, 
#                                6: 27.972027972027973, 7: 122.13740458015268, 8: 1.0}
######################################
######################################
# Classwise_Dist_Degree_6= {0: 9740, 1: 840, 2: 16000, 3: 16000, 4: 959, 5: 531, 6: 292, 7: 70, 8: 16000}
# Classwise_Weightage_Degree_6= {0: 1.6427104722792607, 1: 19.047619047619047, 2: 1.0, 3: 1.0, 4: 16.684045881126174, 5: 30.131826741996232, 
#                                6: 54.794520547945204, 7: 228.57142857142858, 8: 1.0}
######################################
# ALL_CLASSES_WEIGTHAGES = {"3":Classwise_Weightage_Degree_3, "4":Classwise_Weightage_Degree_4,
#                      "5":Classwise_Weightage_Degree_5, "6":Classwise_Weightage_Degree_6}
# ALL_CLASSES_DIST = {"3":Classwise_Dist_Degree_3, "4":Classwise_Dist_Degree_4,
#                      "5":Classwise_Dist_Degree_6, "6":Classwise_Dist_Degree_6}
######################################
# 30000 Classes
# CLASSES_WEIGTHAGES = {0: 1.0, 1: 2.3366192040890836, 2: 1.0, 3: 1.0, 4: 2.2783908864364544, 
#                    5: 2.101116217990808, 6: 8.473080317740513, 7: 12.307692307692308, 8: 1.0}
# CLASSES_WEIGTHAGES_Masked = {"Degree_3_{0}".format("With_Horizon_Masked"):CLASSES_WEIGTHAGES
# }
CLASS_MAPPINGS = []
# for label in range(0, OUTPUT_CLASSES):
#     CLASS_MAPPINGS.append(str(label))
##############################################################################################



##############################################################################################
#TRAINED MODEL IDENTIFIER & PATHS
# MODEL_NAME_PREFIX = None#"Attitude_Trained_Model_{0}".format(With_Horizon)
# MODEL_NAME_PREFIX = "Guages_Trained_Model"
BASE = "."#"/scratch/rotorcraftdata/RU_FAA_Rotorcraft_Safety_Workspace/FAA_Attitude_Estimation"



# DATA PATHS
ROOT_DS_FOLDER_PATH = None#"./Data_{0}/Training Data/Frames_WindSheild_Threshold_{1}_With_Hist_False".format(C_Degree, With_Horizon)#Frames_WindSheild_Threshold_3_With_Hist_False/"
# ROOT_DS_FOLDER_PATH = "/home/hikmat/Desktop/DLWSpace/FAA/Data_Without_Horizon_Masked/Frames_WindSheild_Threshold_{0}_With_Hist_False/".format(
#     C_Degree)#/Frames_WindSheild_Threshold_3_With_Hist_False"
TRAINING_DIRECTORY_PATH = None
VALIDATION_DIRECTORY_PATH = None
TESTING_DIRECTORY_PATH = None
Store_Trained_ML_Model_Info_At = None
##############################################################################################



##############################################################################################
# MODEL SELECTION HELP OR AVAILABLE MODELS
class Available_Models(Enum):
    VGG16 = "VGG16"
    VGG19 = "VGG19"
    InceptionV3 = "InceptionV3"
    Xception = "Xception" # TensorFlow ONLY
    ResNet50 = "ResNet50"
    ResNet18 = "ResNet18"
    ResNet34 = "ResNet34"
    DenseNet121 = "DenseNet121"
    DenseNet169 = "DenseNet169"
    DenseNet201 = "DenseNet201"
    MobileNet = "MobileNet"
    InceptionResNetV2 = "InceptionResNetV2"
    NASNetLarge = "NASNetLarge"
    NASNetMobile = "NASNetMobile"
    Lenet_Style = "Lenet_Style"
##############################################################################################    



AVAILABLE_GPUs= 1


In [7]:
IS_CLASSES_WEIGTHAGES = False
Is_To_Add_Dropout = True#True
LEARNING_RATE_LST = [0.0001]#, 0.005, 0.009]
Dropout_Rate_LST = [0.25]#, 0.5, 0.0]
Dense_Layers_Neurons = []#[512]
LEARNING_RATE = None
Dropout_Rate = None
TEST_CSV_PATH = None
TEST_CSV_Predicted_PATH = None
TEST_CSV = None
Is_To_Apply_Data_Augmentation = False
Is_To_Apply_Data_Augmentation_Lt=[False]

# Classwise_Dist_Degree_3= {0: 16000, 1: 8217, 2: 16000, 3: 16000, 4: 8427, 5: 9138, 6: 2266, 7: 1560, 8: 16000}
Classwise_Weightage_Degree_3= {0: 1.0, 1: 1.9471826700742363, 2: 1.0, 3: 1.0, 4: 1.8986590720303786, 5: 1.7509301816590064, 
                               6: 7.0609002647837595, 7: 10.256410256410257, 8: 1.0}

######################################
######################################
# Classwise_Dist_Degree_4= {0: 16000, 1: 3034, 2: 16000, 3: 16000, 4: 4486, 5: 3702, 6: 953, 7: 417, 8: 16000}
Classwise_Weightage_Degree_4= {0: 1.0, 1: 5.273566249176005, 2: 1.0, 3: 1.0, 4: 3.5666518056174765, 5: 4.321988114532685, 
                               6: 16.7890870933893, 7: 38.36930455635492, 8: 1.0}
######################################
######################################
# Classwise_Dist_Degree_5= {0: 14316, 1: 1502, 2: 16000, 3: 16000, 4: 2059, 5: 1448, 6: 572, 7: 131, 8: 16000}
Classwise_Weightage_Degree_5= {0: 11.1176306230790725, 1: 15.652463382157125, 2: 1.0, 3: 1.0, 
                               4: 17.770762506070908, 5: 210.049723756906078, 
                               6: 27.972027972027973, 7: 122.13740458015268, 8: 1.0}
######################################



REGULARIZE_EARLY_LAYERS_WIHT_PERCENT = 1
REGULARIZE_EARLY_LAYERS_WITH_RATIO = 0.001


Dell_Path = ".//home/hikmat/Desktop/DLWSpace/FAA_Guages_Data/" # If training on the local machine
# Dell_Path = "." # If training on HPC
if __name__ == "__main__":
    global LEARNING_RATE
    global Dropout_Rate
    
    global INPUT_SIZE
    global BATCH_SIZE 
    global ROOT_DS_FOLDER_PATH
    global TRAINING_DIRECTORY_PATH
    global VALIDATION_DIRECTORY_PATH
#     global CLASSES_WEIGTHAGES
    global CLASS_MAPPINGS 
    global OUTPUT_CLASSES
#     global TEST_CSV_PATH
#     global C_Degree
#     global C_With_Horizon
#     global MODEL_NAME_PREFIX
    global Store_Trained_ML_Model_Info_At
#     global TEST_CSV
#     global TEST_CSV_Predicted_PATH
#     global Is_To_Apply_Data_Augmentation



#     Dell_Guages = ["NR(%)"]
#     Rutger_Guages = ["N1_NG 1(%)","N1_NG 2(%)",]
#     GCP_Guages = ["Pitch Angle(deg)"]#"True Airspeed(kn)"]#True Airspeed(kn)"]#"Pitch Angle(deg)"]#"True Airspeed(kn)"]
#     Gauges = GCP_Guages#["NR(%)", "Torque 1(%)", "Torque 2(%)"]#,"True Airspeed(kn)"]#,"Latitude(deg)","Longitude(deg)","NR(%)","Torque 1(%)",

    print("AVAILABLE_GPUs = ", AVAILABLE_GPUs)
    for learning_rate, dropout_rate, l_Is_To_Apply_Data_Augmentation in zip(LEARNING_RATE_LST, 
                                                Dropout_Rate_LST, Is_To_Apply_Data_Augmentation_Lt):
        LEARNING_RATE = learning_rate
        Dropout_Rate = dropout_rate
        Is_To_Apply_Data_Augmentation = l_Is_To_Apply_Data_Augmentation
        for guage in Gauges:
            C_Guage = guage

#             Trained_Models_Root_Path = "./Trained_Models_of_Guage_With_Learning_Rate_{0}_With_Dropout_{1}_Ratio_{2}_With_D_Aug_{3}".format(
#             LEARNING_RATE,Is_To_Add_Dropout, Dropout_Rate, Is_To_Apply_Data_Augmentation)
            Trained_Models_Root_Path = "./Trained_Models"
            if(os.path.exists(Trained_Models_Root_Path)):
                pass
            else:
                os.makedirs(Trained_Models_Root_Path)

            Store_Trained_ML_Model_Info_At = "{0}/{1}".format(Trained_Models_Root_Path, C_Guage)    
            if(os.path.exists(Store_Trained_ML_Model_Info_At)):
                pass
            else:
                os.makedirs(Store_Trained_ML_Model_Info_At)
                
            print("*" * 50)
            print("*" * 25)
            print("c_guage={0}".format(C_Guage))
    #         print("C_With_Horizon={0}".format(C_With_Horizon))
    #         if(Is_To_Apply_Class_Weigtage is False):
    #             CLASSES_WEIGTHAGES = None
    #             print("Is_To_Apply_Class_Weigtage={0}".format(Is_To_Apply_Class_Weigtage))
    #         else:
    #             print("Is_To_Apply_Class_Weigtage={0}".format(Is_To_Apply_Class_Weigtage))
    #             CLASSES_WEIGTHAGES = ALL_CLASSES_WEIGTHAGES[str(C_Degree)]
    #             continue
    #         print("CLASSES_WEIGTHAGES={0}".format(CLASSES_WEIGTHAGES))

    
    #             ROOT_DS_FOLDER_PATH = "/home/hikmat/Desktop/DLWSpace/FAA/Data_{0}/Training Data/Frames_WindSheild_Threshold_{1}_With_Hist_False".format(
    #                 C_With_Horizon, C_Degree)#Frames_WindSheild_Threshold_3_With_Hist_False/"
#             if(Is_Running_On_Local_Desktop):
            if(IS_CLASSES_WEIGTHAGES):
                CLASSES_WEIGTHAGES = Classwise_Weightage_Degree_3
            else:
                CLASSES_WEIGTHAGES = None
            
            ROOT_DS_FOLDER_PATH = "{1}/Training/Training Data/{0}_Data".format(
            C_Guage, Dell_Path)
            TEST_CSV_PATH  = "{1}/Training/CSV/{0}_ML_CSV_For_CNN_Test.csv".format(C_Guage, Dell_Path)
            TEST_CSV_Predicted_PATH = "{1}/Training/CSV/{0}_ML_Predicted_CSV_For_CNN_Test_{2}.csv".format(
                C_Guage, Dell_Path, str(datetime.now()))
#             elif(FAA_HPC):
#                 ROOT_DS_FOLDER_PATH = "{1}/Training/Training Data/{0}_Data".format(
#                 C_Guage, BASE)
#                 #TEST_CSV_PATH  = "{0}/Training/CSV/{1}_ML_Predicted_CSV_For_CNN_Test.csv".format(BASE, C_Guage)
#                 #TEST_CSV_Predicted_PATH = "./Training/CSV/{0}_ML_CSV_For_CNN_Test.csv".format(C_Guage)
#             else:
#                 ROOT_DS_FOLDER_PATH = "./Training/Training Data/{0}_Data".format(
#                 C_Guage)
#                 #TEST_CSV_PATH  = "./Training/CSV/{0}_ML_CSV_For_CNN_Test.csv".format(C_Guage)
#                 #TEST_CSV_Predicted_PATH = "./Training/CSV/{0}_ML_Predicted_CSV_For_CNN_Test.csv".format(C_Guage)


    #             True Airspeed(kn)_ML_CSV_For_CNN_Test

            TRAINING_DIRECTORY_PATH = "{0}/TRAINING_DIRECTORY".format(ROOT_DS_FOLDER_PATH)
            OUTPUT_CLASSES = len(os.listdir(TRAINING_DIRECTORY_PATH))
            CLASS_MAPPINGS = []
            for label in range(0, OUTPUT_CLASSES):
    #             print("Label=", label)
                CLASS_MAPPINGS.append(str(label))
            VALIDATION_DIRECTORY_PATH = "{0}/VALIDATION_DIRECTORY".format(ROOT_DS_FOLDER_PATH)
#             TESTING_DIRECTORY_PATH = "{0}/TESTING_DIRECTORY".format(ROOT_DS_FOLDER_PATH)

            print("Current TRAINING_DIRECTORY_PATH={0}".format(TRAINING_DIRECTORY_PATH))
            print("Current VALIDATION_DIRECTORY_PATH={0}".format(VALIDATION_DIRECTORY_PATH))
#             print("Current TESTING_DIRECTORY_PATH={0}".format(TESTING_DIRECTORY_PATH))
#             print("PREDICTED_CSV_WILL_BE_SAVED_AT_PATH=" + str(TEST_CSV_Predicted_PATH))
#             print("TEST_CSV_PATH={0}".format(TEST_CSV_PATH))
#             TEST_CSV = pd.read_csv(TEST_CSV_PATH)
            print("*" * 25)




            
#             BATCH_SIZE = 96 * AVAILABLE_GPUs
#             INPUT_SIZE = (224,224, 3)
#             SELECTED_MODEL = Available_Models.ResNet50
#             Initiate_Model_Training()

            
#             BATCH_SIZE = 1024 * AVAILABLE_GPUs
#             INPUT_SIZE = (64,64, 3)
#             SELECTED_MODEL = Available_Models.Lenet_Style
#             Initiate_Model_Training()
          

            
            
#             BATCH_SIZE = 512 * AVAILABLE_GPUs
#             INPUT_SIZE = (128,128, 3)
#             SELECTED_MODEL = Available_Models.ResNet18
#             Initiate_Model_Training()
#             print("Saving Prediction of {0}".format(SELECTED_MODEL))
# #             TEST_CSV.to_csv(TEST_CSV_Predicted_PATH)

#             BATCH_SIZE = 512 * AVAILABLE_GPUs
#             INPUT_SIZE = (128,128, 3)
#             SELECTED_MODEL = Available_Models.VGG19
#             Initiate_Model_Training()
#             print("Saving Prediction of {0}".format(SELECTED_MODEL))
# #             TEST_CSV.to_csv(TEST_CSV_Predicted_PATH)
            
            
            #BATCH_SIZE = 512 * AVAILABLE_GPUs
            #INPUT_SIZE = (128,128, 3)
            #SELECTED_MODEL = Available_Models.ResNet34
            #Initiate_Model_Training()
            #print("Saving Prediction of {0}".format(SELECTED_MODEL))
            #TEST_CSV.to_csv(TEST_CSV_Predicted_PATH)
            
            
            
            BATCH_SIZE = 96 * AVAILABLE_GPUs
            INPUT_SIZE = (224,224, 3)
            SELECTED_MODEL = Available_Models.ResNet50
            Initiate_Model_Training()
            print("Saving Prediction of {0}".format(SELECTED_MODEL))
#             TEST_CSV.to_csv(TEST_CSV_Predicted_PATH)
   

            

            
            
#             BATCH_SIZE = 32 * AVAILABLE_GPUs
#             INPUT_SIZE = (299,299, 3)
#             SELECTED_MODEL = Available_Models.Xception
#             Initiate_Model_Training()
#             print("Saving Prediction of {0}".format(SELECTED_MODEL))
            

#             BATCH_SIZE = 32 * AVAILABLE_GPUs
#             INPUT_SIZE = (299,299, 3)
#             SELECTED_MODEL = Available_Models.InceptionV3
#             Initiate_Model_Training()

#             BATCH_SIZE = 32 * AVAILABLE_GPUs
#             INPUT_SIZE = (299,299, 3)
#             SELECTED_MODEL = Available_Models.InceptionResNetV2
#             Initiate_Model_Training()

#             BATCH_SIZE = 32 * AVAILABLE_GPUs
#             INPUT_SIZE = (224,224, 3)
#             SELECTED_MODEL = Available_Models.DenseNet121
#             Initiate_Model_Training()

#             BATCH_SIZE = 96 * AVAILABLE_GPUs
#             INPUT_SIZE = (224,224, 3)
#             SELECTED_MODEL = Available_Models.VGG16
#             Initiate_Model_Training()
            
#             print("SAVING_PREDICTED_CSV_TO_PATH=" + str(TEST_CSV_Predicted_PATH))
#             TEST_CSV.to_csv(TEST_CSV_Predicted_PATH)
#             TEST_CSV = None
#             TEST_CSV_PATH = None
#             TEST_CSV_Predicted_PATH = None
            print("*" * 25)
            print("Gauge {0} Training Has Completed!".format(C_Guage))
#         break


AVAILABLE_GPUs =  1
**************************************************
*************************
c_guage=True Airspeed(kn)
Current TRAINING_DIRECTORY_PATH=/home/hikmat/Desktop/DLWSpace/FAA_Guages_Data//Training/Training Data/True Airspeed(kn)_Data/TRAINING_DIRECTORY
Current VALIDATION_DIRECTORY_PATH=/home/hikmat/Desktop/DLWSpace/FAA_Guages_Data//Training/Training Data/True Airspeed(kn)_Data/VALIDATION_DIRECTORY
*************************
SELECTED MODEL #Available_Models.ResNet50


/home/hikmat/anaconda3/envs/KerasWithTFPY35/lib/python3.5/site-packages/keras_applications/resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Regularize 177 early layers
Zero layers freezed
Compiling single GPU model
Found 595054 images belonging to 152 classes.
TRAINING_DIRECTORY_PATH= /home/hikmat/Desktop/DLWSpace/FAA_Guages_Data//Training/Training Data/True Airspeed(kn)_Data/TRAINING_DIRECTORY
TRAIN_GENERATOR.class_indices = {'98': 98, '132': 132, '78': 78, '56': 56, '150': 150, '39': 39, '144': 144, '118': 118, '129': 129, '1': 1, '31': 31, '109': 109, '138': 138, '58': 58, '24': 24, '105': 105, '52': 52, '90': 90, '13': 13, '9': 9, '102': 102, '149': 149, '2': 2, '134': 134, '12': 12, '66': 66, '65': 65, '126': 126, '103': 103, '147': 147, '77': 77, '121': 121, '61': 61, '95': 95, '70': 70, '131': 131, '63': 63, '57': 57, '3': 3, '32': 32, '72': 72, '143': 143, '94': 94, '26': 26, '108': 108, '79': 79, '119': 119, '135': 135, '128': 128, '45': 45, '47': 47, '137': 137, '136': 136, '122': 122, '10': 10, '87': 87, '140': 140, '48': 48, '133': 133, '139': 139, '97': 97, '40': 40, '35': 35, '38': 38, '86': 86, '74': 74, '75

Epoch 1/500
6198/6198 [==============================] - 6484s 1s/step - loss: 4.9610 - acc: 0.6371 - val_loss: 3.4574 - val_acc: 0.6299

Epoch 00001: val_acc improved from -inf to 0.62987, saving model to ./Trained_Models/True Airspeed(kn)/CheckPoint_Available_Models.ResNet50_True Airspeed(kn).hdf5
Epoch 2/500
6198/6198 [==============================] - 6480s 1s/step - loss: 2.5220 - acc: 0.7844 - val_loss: 2.6269 - val_acc: 0.6459

Epoch 00002: val_acc improved from 0.62987 to 0.64593, saving model to ./Trained_Models/True Airspeed(kn)/CheckPoint_Available_Models.ResNet50_True Airspeed(kn).hdf5
Epoch 3/500
6198/6198 [==============================] - 6479s 1s/step - loss: 1.9038 - acc: 0.8220 - val_loss: 2.2364 - val_acc: 0.6672

Epoch 00003: val_acc improved from 0.64593 to 0.66725, saving model to ./Trained_Models/True Airspeed(kn)/CheckPoint_Available_Models.ResNet50_True Airspeed(kn).hdf5
Epoch 4/500
6198/6198 [==============================] - 6480s 1s/step - loss: 1.6039 - acc

Found 78445 images belonging to 107 classes.
VALIDATION_DIRECTORY_PATH= /home/hikmat/Desktop/DLWSpace/FAA_Guages_Data//Training/Training Data/NR(%)_Data/VALIDATION_DIRECTORY
TRAIN_GENERATOR.class_indices = {'98': 98, '78': 78, '56': 56, '39': 39, '1': 1, '31': 31, '58': 58, '24': 24, '105': 105, '52': 52, '90': 90, '13': 13, '9': 9, '102': 102, '2': 2, '12': 12, '66': 66, '65': 65, '103': 103, '77': 77, '61': 61, '95': 95, '70': 70, '63': 63, '57': 57, '3': 3, '32': 32, '72': 72, '94': 94, '26': 26, '79': 79, '45': 45, '47': 47, '10': 10, '87': 87, '48': 48, '97': 97, '40': 40, '35': 35, '38': 38, '86': 86, '74': 74, '75': 75, '11': 11, '15': 15, '64': 64, '44': 44, '71': 71, '83': 83, '91': 91, '82': 82, '50': 50, '34': 34, '41': 41, '89': 89, '42': 42, '5': 5, '59': 59, '67': 67, '16': 16, '88': 88, '93': 93, '92': 92, '4': 4, '54': 54, '106': 106, '99': 99, '8': 8, '21': 21, '19': 19, '0': 0, '100': 100, '14': 14, '80': 80, '20': 20, '62': 62, '7': 7, '29': 29, '76': 76, '73': 73, '

Epoch 1/500
927/927 [==============================] - 1226s 1s/step - loss: 5.9732 - acc: 0.8832 - val_loss: 5.5381 - val_acc: 0.8695

Epoch 00001: val_acc improved from -inf to 0.86950, saving model to ./Trained_Models/NR(%)/CheckPoint_Available_Models.ResNet50_NR(%).hdf5
Epoch 2/500
927/927 [==============================] - 1210s 1s/step - loss: 4.7597 - acc: 0.9599 - val_loss: 4.5656 - val_acc: 0.8574

Epoch 00002: val_acc did not improve from 0.86950
Epoch 3/500
927/927 [==============================] - 1212s 1s/step - loss: 3.6836 - acc: 0.9685 - val_loss: 3.5179 - val_acc: 0.8700

Epoch 00003: val_acc improved from 0.86950 to 0.86997, saving model to ./Trained_Models/NR(%)/CheckPoint_Available_Models.ResNet50_NR(%).hdf5
Epoch 4/500
927/927 [==============================] - 1200s 1s/step - loss: 2.7808 - acc: 0.9720 - val_loss: 2.7320 - val_acc: 0.8664

Epoch 00004: val_acc did not improve from 0.86997
Epoch 5/500
927/927 [==============================] - 1200s 1s/step - loss

Epoch 1/500
3394/3394 [==============================] - 3728s 1s/step - loss: 5.0143 - acc: 0.8717 - val_loss: 3.4556 - val_acc: 0.8711

Epoch 00001: val_acc improved from -inf to 0.87112, saving model to ./Trained_Models/N1_NG 1(%)/CheckPoint_Available_Models.ResNet50_N1_NG 1(%).hdf5
Epoch 2/500
3394/3394 [==============================] - 3723s 1s/step - loss: 2.3851 - acc: 0.9232 - val_loss: 2.0000 - val_acc: 0.8718

Epoch 00002: val_acc improved from 0.87112 to 0.87177, saving model to ./Trained_Models/N1_NG 1(%)/CheckPoint_Available_Models.ResNet50_N1_NG 1(%).hdf5
Epoch 3/500
3394/3394 [==============================] - 3722s 1s/step - loss: 1.4815 - acc: 0.9361 - val_loss: 1.4978 - val_acc: 0.8860

Epoch 00003: val_acc improved from 0.87177 to 0.88598, saving model to ./Trained_Models/N1_NG 1(%)/CheckPoint_Available_Models.ResNet50_N1_NG 1(%).hdf5
Epoch 4/500
3394/3394 [==============================] - 3722s 1s/step - loss: 1.0869 - acc: 0.9460 - val_loss: 1.2085 - val_acc: 0.8

Epoch 1/500
3124/3124 [==============================] - 3449s 1s/step - loss: 5.0967 - acc: 0.8716 - val_loss: 3.6917 - val_acc: 0.8516

Epoch 00001: val_acc improved from -inf to 0.85164, saving model to ./Trained_Models/N1_NG 2(%)/CheckPoint_Available_Models.ResNet50_N1_NG 2(%).hdf5
Epoch 2/500
3124/3124 [==============================] - 3444s 1s/step - loss: 2.4960 - acc: 0.9259 - val_loss: 2.0571 - val_acc: 0.8808

Epoch 00002: val_acc improved from 0.85164 to 0.88081, saving model to ./Trained_Models/N1_NG 2(%)/CheckPoint_Available_Models.ResNet50_N1_NG 2(%).hdf5
Epoch 3/500
3124/3124 [==============================] - 3453s 1s/step - loss: 1.5161 - acc: 0.9396 - val_loss: 1.6229 - val_acc: 0.8605

Epoch 00003: val_acc did not improve from 0.88081
Epoch 4/500
3124/3124 [==============================] - 3468s 1s/step - loss: 1.1046 - acc: 0.9477 - val_loss: 1.2787 - val_acc: 0.8822

Epoch 00004: val_acc improved from 0.88081 to 0.88217, saving model to ./Trained_Models/N1_NG 2(%

# INDEPENDENT EVALUATION ON TESTSET

In [8]:
# C_Guage = "NR(%)"128
# LEARNING_RATE = 0.001
# Dropout_Rate = 0.1
# Model_Name = "ResNet50"
# Root_Path = "/home/hikmat/Desktop/DLWSpace/FAA_Guages_Data/AHS_Github/AHS/Trained_Models_of_Guage_With_Learning_Rate_{0}_With_Dropout_True_Ratio_{1}/{2}".format(
#     LEARNING_RATE, Dropout_Rate, C_Guage)
# Model = None
# Model_Path ="{0}/{1}".format(Root_Path, "Guages_Trained_Model_Available_Models.{0}_Degree_{1}_model_architecture".format(Model_Name, C_Guage))

# Model_Weights ="{0}/{1}".format(Root_Path, "CheckPoint_Guages_Trained_Model_Available_Models.{0}_Degree_{1}_Best_Model_Weights".format(Model_Name, C_Guage))

# Model = load_model_and_weights_from_path(model_path=Model_Path,
#                                          weights_paths=Model_Weights)
# TEST_CSV_I = pandas.read_csv(
#     "/home/hikmat/Desktop/DLWSpace/FAA_Guages_Data/Training/CSV/{0}_ML_CSV_For_CNN_Test.csv".format(C_Guage))
# TEST_CSV_I_OutPut_Path = "/home/hikmat/Desktop/DLWSpace/FAA_Guages_Data/Training/CSV/{0}_{1}_ML_Prd_CSV_For_CNN_Test.csv".format(C_Guage,Model_Name)
# INPUT_SIZE = (224, 224, 3)






In [9]:
# TEST_CSV_I = TestSet_Evaluate_Model(TEST_CSV= TEST_CSV_I, 
#                        Model=Model,Model_Name=Model_Name, C_Guage=C_Guage,
#                        INPUT_SIZE=INPUT_SIZE)
# TEST_CSV_I.to_csv(TEST_CSV_I_OutPut_Path)

In [10]:
# TEST_CSV_I.head(5)

In [11]:
# str(datetime.now())